In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__

'1.4.1'

In [4]:
tf.keras.__version__

'2.0.8-tf'

#  加载数据

我们将使用由IMDB电影评论组成的50000个数据集。 Keras有一个内置的功能来下载一个类似的数据集（但显然是一半的大小）。 然而，Keras的版本已经将数据集中的文本转换为整数标记，这是使用自然语言工作的关键部分，本教程中还将演示这些语言，因此我们下载了实际的文本数据。

注意：数据集为84 MB，将自动下载。

In [5]:
import imdb

如果要将文件保存在其他目录中，请更改此项。

In [6]:
# imdb.data_dir = "data/IMDB/"

自动下载并提取文件。

In [7]:
imdb.maybe_download_and_extract()

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


加载训练和测试集。

In [8]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [9]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


将下面的一些用途组合成一个数据集。

In [10]:
data_text = x_train_text + x_test_text

从训练集打印一个例子，看看数据看起来是否正确。

In [11]:
x_train_text[1]

"There are not many movies around that have given me a feeling like Stardust did all throughout the course of the film. As magically fairy-tale-like as The Princess Bride, Stardust is most definitely the most wonderful fantasy spectacle of the 2000's as well as the 1990's. Exciting, hilarious and equipped with wonderful imagery as well as unforgettable characters, Michelle Pfeiffer and Robert DeNiro's especially, I challenge anyone to watch this movie without a smile. From the first ten minutes of the film you know perfectly well how it will end, but it is the journey and not the destination that enthralls the viewer from start to finish.<br /><br />Ten stars, and not a decimal less."

真正的“class”是电影评论的一种观点。 消极情绪为0.0，积极情绪为1.0。 在这种情况下，评论是积极的。

In [12]:
y_train[1]

1.0

# 标记生成器
神经网络不能直接在文本字符串上工作，所以我们必须以某种方式对其进行转换。 这种转换有两个步骤，第一步称为“标记器”，它将单词转换为整数，并在输入到神经网络之前在数据集上完成。 第二步是神经网络本身的一个组成部分，被称为“嵌入”层，这在下面进一步描述。

我们可能会指示标记器仅使用例如 数据集中最流行的10000个单词。

In [13]:
num_words = 10000 

In [14]:
tokenizer = Tokenizer(num_words=num_words)

标记器然后可以“拟合”到数据集。 这将扫描所有文本，并将其从不需要的字符（如标点符号）中除去，并将其转换为小写字符。 然后，令牌生成器将创建一个包含所有唯一字的词汇表以及用于访问数据的各种数据结构。

请注意，我们在整个数据集上安装了标记器，以便从训练数据和测试数据中收集单词。 这是好的，因为我们只是建立一个词汇表，并希望它尽可能地完整。 实际的神经网络当然只能在训练集上进行训练。

In [15]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 6.93 s, sys: 16 ms, total: 6.95 s
Wall time: 6.95 s


如果您想使用整个词汇表，然后在上面设置num_words = None，那么它会自动设置为词汇大小。 （这是因为Keras的执行有些尴尬。）

In [16]:
if num_words is None:
    num_words = len(tokenizer.word_index)

然后我们可以检查分词器收集的词汇。 这是按数据集中单词的出现次数排序的。 这些整数称为词索引或“标记”，因为它们唯一标识词汇表中的每个词。

In [18]:
tokenizer.word_index

{"'rabid": 60582,
 'elliptic': 67416,
 'angelic': 16008,
 'galactic': 23852,
 "'breakin'": 85947,
 'suchen': 117451,
 'leads\x97gino': 82209,
 'jewels': 11097,
 'avp': 17115,
 'wipes': 13631,
 'roars': 19612,
 'unscience': 97455,
 'couching': 106044,
 'smerdjakov': 119078,
 'keeanu': 67172,
 'rustling': 39529,
 "five'": 89293,
 'parlay': 111397,
 'unencumbered': 60276,
 'perchance': 54104,
 'council': 11242,
 'wilting': 54069,
 'beehives': 51778,
 'wings': 5322,
 "days''": 108992,
 'techy': 66799,
 'tears': 1724,
 'drudgery': 32425,
 'hyperkinesis': 100706,
 'irreligious': 76971,
 'starker': 66712,
 'slevin': 48799,
 'public': 1071,
 'pugninella': 118638,
 'magowan': 62124,
 "'shouldn't": 90427,
 'srathairn': 73837,
 'kettlewell': 102492,
 'dutton': 27686,
 'fingerpainting': 120405,
 'zurn': 110488,
 'salò': 120180,
 'uncontrollable': 15724,
 "stoker's": 42114,
 'vive': 30165,
 'almighty': 6635,
 'abishek': 118762,
 'straightforward': 5130,
 'stingy': 28719,
 'psych': 28476,
 "moviegoe

然后我们可以使用标记器将训练集中的所有文本转换为这些标记的列表。

In [19]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

例如，以下是训练集中的文本：

In [20]:
x_train_text[1]

"There are not many movies around that have given me a feeling like Stardust did all throughout the course of the film. As magically fairy-tale-like as The Princess Bride, Stardust is most definitely the most wonderful fantasy spectacle of the 2000's as well as the 1990's. Exciting, hilarious and equipped with wonderful imagery as well as unforgettable characters, Michelle Pfeiffer and Robert DeNiro's especially, I challenge anyone to watch this movie without a smile. From the first ten minutes of the film you know perfectly well how it will end, but it is the journey and not the destination that enthralls the viewer from start to finish.<br /><br />Ten stars, and not a decimal less."

该文本对应于以下令牌列表：文本转成向量

In [21]:
np.array(x_train_tokens[1])

array([  46,   23,   21,  106,   97,  183,   12,   25,  358,   68,    3,
        558,   37,  115,   29,  474,    1,  265,    4,    1,   19,   14,
       7658, 3065,  793,   37,   14,    1, 2761, 3655,    6,   88,  406,
          1,   88,  393, 1029, 6088,    4,    1,   14,   69,   14,    1,
       8602, 1141,  578,    2,   16,  393, 2657,   14,   69,   14, 3202,
        102, 2917, 6187,    2,  610,  261,   10, 3029,  250,    5,  103,
         11,   17,  208,    3, 1861,   36,    1,   86,  713,  228,    4,
          1,   19,   22,  118,  922,   69,   85,    9,   80,  127,   18,
          9,    6,    1, 1261,    2,   21,    1, 6303,   12,    1,  536,
         36,  375,    5, 1391,    7,    7,  713,  405,    2,   21,    3,
        340])

In [22]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

# 填充和截断数据
递归神经网络可以将任意长度的序列作为输入，但为了使用整批数据，序列需要具有相同的长度。 有两种方法可以实现这一点：（A）要么我们确保整个数据集中的所有序列具有相同的长度，要么（B）我们编写一个自定义数据生成器，以确保序列在每个批次中具有相同的长度。

解决方案（A）更简单，但如果我们使用数据集中最长序列的长度，那么我们浪费了大量内存。 这对于大型数据集尤为重要。

所以为了做出妥协，我们将使用涵盖数据集中大部分序列的序列长度，然后我们将截断较长的序列并填充较短的序列。

首先我们计算数据集中所有序列中的令牌数量。

In [23]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

序列中的平均令牌数为：

In [24]:
np.mean(num_tokens)

221.27716000000001

序列中令牌的最大数量为：

In [25]:
np.max(num_tokens)

2209

我们允许的令牌的最大数量设置为平均值加2个标准差。

In [26]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

这涵盖了约95％的数据集。

In [27]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94532000000000005

当填充或截断具有不同长度的序列时，我们需要确定是否要填充或截断“pre”或“post”。 如果序列被截断，则意味着序列的一部分被简单地丢弃。 如果序列被填充，则表示将零添加到序列中。

所以'pre'或'post'的选择可能很重要，因为它决定了我们是否在截断时丢弃了序列的第一部分或最后一部分，并决定了在填充时是否将序列的零或开头添加到序列的末尾。 这可能会混淆循环神经网络。

In [28]:
pad = 'pre'

In [29]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [30]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

现在我们已经将训练集转换成了一个具有这种形状的整数（令牌）的大矩阵：

In [31]:
x_train_pad.shape

(25000, 544)

测试集的矩阵具有相同的形状：

In [32]:
x_test_pad.shape

(25000, 544)

例如，我们在上面有以下令牌序列：

In [33]:
np.array(x_train_tokens[1])

array([  46,   23,   21,  106,   97,  183,   12,   25,  358,   68,    3,
        558,   37,  115,   29,  474,    1,  265,    4,    1,   19,   14,
       7658, 3065,  793,   37,   14,    1, 2761, 3655,    6,   88,  406,
          1,   88,  393, 1029, 6088,    4,    1,   14,   69,   14,    1,
       8602, 1141,  578,    2,   16,  393, 2657,   14,   69,   14, 3202,
        102, 2917, 6187,    2,  610,  261,   10, 3029,  250,    5,  103,
         11,   17,  208,    3, 1861,   36,    1,   86,  713,  228,    4,
          1,   19,   22,  118,  922,   69,   85,    9,   80,  127,   18,
          9,    6,    1, 1261,    2,   21,    1, 6303,   12,    1,  536,
         36,  375,    5, 1391,    7,    7,  713,  405,    2,   21,    3,
        340])

这只是填充以创建以下序列。 请注意，当它被输入到递归神经网络时，它首先输入很多零。 如果我们填充了'post'，那么它会首先输入整数标记，然后输入很多零。 这可能会混淆循环神经网络。

In [34]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# Tokenizer反向映射
出于某种奇怪的原因，Keras实现一个标记器似乎没有将整数标记反向映射回单词，这需要从令牌列表重建文本字符串。 所以我们在这里做这个映射。

In [35]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

辅助函数用于将令牌列表转换回单词串。

In [36]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

例如，这是来自数据集的原始文本：

In [37]:
x_train_text[1]

"There are not many movies around that have given me a feeling like Stardust did all throughout the course of the film. As magically fairy-tale-like as The Princess Bride, Stardust is most definitely the most wonderful fantasy spectacle of the 2000's as well as the 1990's. Exciting, hilarious and equipped with wonderful imagery as well as unforgettable characters, Michelle Pfeiffer and Robert DeNiro's especially, I challenge anyone to watch this movie without a smile. From the first ten minutes of the film you know perfectly well how it will end, but it is the journey and not the destination that enthralls the viewer from start to finish.<br /><br />Ten stars, and not a decimal less."

我们可以通过将标记列表转换回单词来重新创建除标点符号和其他符号之外的文本：

In [38]:
tokens_to_string(x_train_tokens[1])

"there are not many movies around that have given me a feeling like did all throughout the course of the film as magically fairy tale like as the princess bride is most definitely the most wonderful fantasy spectacle of the as well as the 1990's exciting hilarious and with wonderful imagery as well as unforgettable characters michelle pfeiffer and robert especially i challenge anyone to watch this movie without a smile from the first ten minutes of the film you know perfectly well how it will end but it is the journey and not the destination that the viewer from start to finish br br ten stars and not a less"

# 创建循环神经网络
我们现在准备创建循环神经网络（RNN）。 由于其简单性，我们将使用Keras API。 有关Keras的教程，请参见教程＃03-C。

In [39]:
model = Sequential()

RNN中的第一层是所谓的嵌入层，其将每个整数标记转换为值向量。 这是必要的，因为对于10000字的词汇表，整数标记可以取0到10000之间的值。 RNN无法处理如此广泛的数值。 嵌入层被训练为RNN的一部分，并将学习将具有相似语义含义的单词映射到相似的嵌入向量，如下面将进一步示出的那样。

首先我们为每个整数标记定义嵌入向量的大小。 在这种情况下，我们将其设置为8，以便每个整数标记都将转换为长度为8的矢量。嵌入矢量的值通常大致落在-1.0和1.0之间，尽管它们可能会略微超过这些值。

嵌入向量的大小通常在100-300之间选择，但对于情感分析来说，它似乎可以很好地工作。

In [40]:
embedding_size = 8

嵌入层还需要知道词汇表（num_words）中的单词数量和填充的标记序列（max_tokens）的长度。 我们也给这个图层一个名字，因为我们需要在下面进一步检索它的权重。

In [41]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

我们现在可以将第一个门控循环单元（GRU）添加到网络中。 这将有16个输出。 因为我们将在此之后添加第二个GRU，所以我们需要返回数据序列，因为下一个GRU需要将序列作为其输入。

In [42]:
model.add(GRU(units=16, return_sequences=True))

这增加了具有8个输出单元的第二个GRU。 接下来是另一个GRU，所以它也必须返回序列。

In [43]:
model.add(GRU(units=8, return_sequences=True))

这增加了第三个也是最后一个GRU，具有4个输出单位。 紧接着是一个密集层，所以它应该只给出GRU的最终输出，而不是一个完整的输出序列。

In [44]:
model.add(GRU(units=4))

添加一个完全连接/密集层，计算0.0到1.0之间的值作为分类输出。

In [45]:
model.add(Dense(1, activation='sigmoid'))

使用具有给定学习率的Adam优化器。

In [47]:
optimizer = Adam(lr=1e-3)

编译Keras模型，以便可以进行训练

In [48]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 16)          1200      
_________________________________________________________________
gru_2 (GRU)                  (None, None, 8)           600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


# 训练循环神经网络
我们现在可以训练模型。 请注意，我们正在使用填充序列的数据集。 我们使用5％的训练集作为一个小的验证集，所以我们对这个模型是否泛化很好或者它是否适合训练集有一个粗略的认识。

In [50]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 435s - loss: 0.6597 - acc: 0.5776 - val_loss: 0.3170 - val_acc: 0.9216
Epoch 2/3
23750/23750 [==============================] - 435s - loss: 0.4068 - acc: 0.8248 - val_loss: 0.2846 - val_acc: 0.8984
Epoch 3/3
23750/23750 [==============================] - 410s - loss: 0.2943 - acc: 0.8852 - val_loss: 0.3789 - val_acc: 0.8504
CPU times: user 30min 26s, sys: 1min 28s, total: 31min 54s
Wall time: 21min 22s


# 性能测试集
现在模型已经过训练，我们可以计算它在测试集上的分类精度。

In [51]:
%%time
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================] - 168s   
CPU times: user 2min 51s, sys: 0 ns, total: 2min 51s
Wall time: 2min 48s


In [52]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 86.74%


# 错误分类的文本示例
为了显示错误分类文本的例子，我们首先计算测试集中前1000个文本的预测情绪。

In [53]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

CPU times: user 6.88 s, sys: 0 ns, total: 6.88 s
Wall time: 6.76 s


这些预测数字在0.0到1.0之间。 我们使用截止点/阈值，并且说0.5以上的所有值取1.0，所有低于0.5的值取0.0。 这给了我们一个0.0或1.0的预测“类”。

In [54]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

测试集中前1000个文本的真实“类”是需要比较的。

In [55]:
cls_true = np.array(y_test[0:1000])

然后，我们可以通过比较这两个数组的所有“类”来获得所有被错误分类的文本的索引。

In [56]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

在使用的1000篇文章中，有多少是错误分类的？

In [57]:
len(incorrect)

94

让我们看看第一个错误分类的文本。 我们会多次使用它的索引。

In [58]:
idx = incorrect[0]
idx

2

错误分类的文字是：

In [59]:
text = x_test_text[idx]
text

'When Liv Ullman\'s character says, "I feel like I\'m in someone else\'s dream and they\'re going to be ashamed when they wake up," she is referring not only to being an unwilling player in society\'s war games, she is referring to being an ignorant participant in life itself. At the film\'s end, when she says that she had a dream that she had a child and she was trying to take care of it, but she forgot something else, the implication is that she has forgotten what she has learned in the war she\'s just survived, that like her own mother before her, she will be unable to pass on any vital lessons to her own child. And, therefore, the cycle of the shame of ignorance will continue...ad infinitum...'

这些是文本预测的和真实的类别：

In [60]:
y_pred[idx]

0.12626585

In [61]:
cls_true[idx]

1.0

# 新数据
让我们尝试分类我们组成的新文本。 其中一些是显而易见的，而另一些则使用否定和讽刺来试图混淆模型，将文本分类错误。

In [62]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

我们首先将这些文本转换为整数标记数组，因为这是模型所需要的。

In [63]:
tokens = tokenizer.texts_to_sequences(texts)

为了将不同长度的文本输入到模型中，我们还需要填充和截断它们。

In [64]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 544)

我们现在可以使用训练好的模型来预测这些文本的情绪。

In [65]:
model.predict(tokens_pad)

array([[ 0.81724364],
       [ 0.67148209],
       [ 0.42932153],
       [ 0.58877438],
       [ 0.39098719],
       [ 0.14605264],
       [ 0.63294291],
       [ 0.20740262]], dtype=float32)

接近0.0的数值意味着负面情绪，接近1.0的数值意味着积极的情绪。 每次训练模型时，这些数字都会有所不同。

# 嵌入
该模型不能直接在整数标记上工作，因为它们的整数值可能介于0和词汇表中的词数之间，例如， 10000.因此，我们需要将整数标记转换为大致介于-1.0和1.0之间的值的向量，这些值可用作神经网络的输入。

从整数标记到实值矢量的映射也称为“嵌入”。 它基本上只是一个矩阵，每行包含单个标记的向量映射。 这意味着我们可以通过简单地使用该标记作为矩阵的索引来快速查找每个整数标记的映射。 在训练过程中，嵌入与模型的其余部分一起学习。

理想情况下，嵌入会学习一个映射，其中意义上相似的单词也具有相似的嵌入值。 让我们来调查这是否发生在这里。

首先，我们需要从模型中获取嵌入层：

In [66]:
layer_embedding = model.get_layer('layer_embedding')

然后我们可以获得嵌入层完成映射的权重。

In [67]:
weights_embedding = layer_embedding.get_weights()[0]

请注意，权重实际上只是一个矩阵，其中词汇表中的词数乘以每个嵌入的向量长度。 这是因为它基本上只是一个查找矩阵。

In [68]:
weights_embedding.shape

(10000, 8)

让我们得到单词'good'的整数标记，它只是词汇表中的一个索引。

In [69]:
token_good = tokenizer.word_index['good']
token_good

49

让我们也得到单词'great'的整数标记。

In [70]:
token_great = tokenizer.word_index['great']
token_great

78

这些integertokens可能会很远，并将取决于数据集中这些单词的频率。

现在让我们比较一下“good”和“great”两个词的矢量嵌入。 其中几个值是相似的，但有些值是完全不同的。 请注意，每次训练模型时，这些值都会改变。

In [71]:
weights_embedding[token_good]

array([ 0.66052264,  0.5165453 ,  0.53161222,  0.49544814,  0.42752418,
        1.19518912,  0.66276532,  0.16886722], dtype=float32)

In [72]:
weights_embedding[token_great]

array([ 0.30738267,  1.4632163 ,  0.41399351,  0.87894005,  0.25113523,
        1.25763977,  0.94943362,  0.04790188], dtype=float32)

同样，我们可以比较“bad”和“horrible”两个词的嵌入。

In [73]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [74]:
weights_embedding[token_bad]

array([ 0.62014925, -0.23285469,  0.77010238, -0.19315891,  1.21860778,
       -0.01226851,  0.52136087,  0.51562059], dtype=float32)

In [75]:
weights_embedding[token_horrible]

array([ 0.98533154, -0.25480568,  0.80838662,  0.36212307,  0.54833591,
       -0.29700345,  0.08748548,  0.53162032], dtype=float32)

# 排序词
我们还可以根据嵌入空间中的“相似性”对词汇表中的所有单词进行排序。 我们想要看看具有相似嵌入向量的词是否也具有相似的含义。

嵌入向量的相似性可以通过不同的度量来度量，例如， 欧几里德距离或余弦距离。

我们有一个帮助函数用于计算这些距离并按排序顺序打印文字。

In [76]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

然后，我们可以根据向量嵌入来打印与“great”这个词相距甚远的单词。 请注意，这些可能会在您每次训练模型时改变。

In [77]:
print_sorted_words('great', metric='cosine')

Distance from 'great':
-0.000 - great
0.009 - tremendous
0.009 - tank
0.019 - sunset
0.022 - hellraiser
0.025 - democracy
0.026 - 1934
0.027 - royal
0.029 - row
0.030 - letterman
...
0.833 - travesty
0.839 - worse
0.848 - pointless
0.880 - horrible
0.886 - mess
0.889 - avoid
0.894 - drags
0.963 - terrible
1.002 - worst
1.098 - waste


类似地，我们可以根据向量嵌入来打印离“worst”一词最近和最远的单词。

In [78]:
print_sorted_words('worst', metric='cosine')

Distance from 'worst':
0.000 - worst
0.051 - incoherent
0.076 - pathetic
0.097 - wasted
0.098 - ridiculous
0.099 - rubbish
0.106 - poor
0.107 - terrible
0.107 - drags
0.108 - mess
...
1.189 - maintained
1.189 - gruff
1.200 - facing
1.201 - 'a
1.210 - 7
1.219 - superb
1.303 - heart
1.345 - worlds
1.398 - favorite
1.501 - excellent


# 结论
本教程展示了使用具有整数标记和嵌入层的递归神经网络进行自然语言处理（NLP）的基本方法。 这被用来对IMDB的电影评论进行情感分析。 如果正确选择超参数，它工作得相当好。 但重要的是要理解这不是人类对文本的理解。 该系统对文本没有任何真正的理解。 这只是一种巧妙的模式识别方式。

# 练习
这些是一些可能有助于提高TensorFlow技能的练习建议。获得TensorFlow的实践经验对于学习如何正确使用它非常重要。
进行任何更改之前，您可能需要备份此笔记本。

- 运行更多的培训时代。它会提高性能吗？
- 如果您的模型适合训练数据，请尝试在GRU中使用丢失图层和丢失。
- 增加或减少词汇表中的单词数量。这在Tokenizer被初始化时完成。它会影响性能吗？
- 将嵌入向量的大小增加到例如200.它会影响性能吗？
- 尝试改变递归神经网络的所有不同的超参数。
- 使用教程＃19中的贝叶斯优化来找到超参数的最佳选择。
- 使用'post'填充和截断pad_sequences（）。它会影响性能吗？
- 使用单个字符而不是标记化词作为词汇表。然后，您可以对每个字符使用单独编码的矢量，而不是使用嵌入图层。
- 使用model.fit_generator（）而不是model.fit（）并创建自己的数据生成器，该数据生成器使用x_train_tokens的随机子集创建一批数据。序列必须填充，以便它们全部匹配最长序列的长度。
- 向朋友解释程序是如何工作的。